In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 사전 Install, import

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 1.7 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 4.9 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595731 sha256=99928f22b7ab923b8a8ac76d88d433bf5ea1b56936aef50eae7e7a4828301d10
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 4.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-r6lo7wit
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-r6lo7wit
     |████████████████████████████████| 129 kB 5.0 MB/s 
     |████████████████████████████████| 54.7 MB 13 kB/s 
     |████████████████████████████████| 4.5 MB 48.3 MB/s 
     |████████████████████████████████| 1.2 MB 19.9 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:41tcmalloc: large alloc 1147494400 bytes == 0x39b6e000 @  0x7f21c514f615 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x58f49e 0x51837f 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4ba70a 0x538136 0x590055 0x51b180 0x5b41c5 0x58f49e 0x51837f 0x5b41c5 0x58f49e 0x51740e 0x58f2a7 0x517947 0x5b41c5 0x58f49e
     |████████████████████████████████| 881.9 MB 1

In [ ]:
import os
import json
import re
import pandas as pd
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook
import pickle

# Data Load

In [ ]:
data = pd.read_excel('/content/drive/MyDrive/DSL/22-2/Modeling/NLP Data.xlsx')

In [ ]:
data.head()

,Review,Label
0,매번 9시간 넘게 자거나 깊게 잠들지 못하고 수시로 깨는데 이 거 켜 놨을 때 딱...,1.0
1,처음엔 휴대폰에서 나는 소리가 적응이 되지 않아 머리가 묵직하고 불편한 데 이틀 ...,2.0
2,그리고 가끔 아침에 일어났을 때 머리가 몽롱 한 데 그 땐 아침밥을 먹거나 커피 ...,1.0
3,단점 가끔 휴대폰 소프트웨어가 밤에 자기 마음대로 업데이트 되는데 그때 폰이 꺼졌...,0.0
4,ㅜㅠ 앱에 표시 되는 광고에 대한 평가 란이 있었는데 그런 거 신경 안 쓰고 잠잘...,2.0


In [ ]:
data = data.astype({'Label' : 'int'})

In [ ]:
data.Label.value_counts()

0    3472
1    3249
2     615
Name: Label, dtype: int64

# Train / Test split -> Train만 증강

## 관련 import

In [ ]:
! git clone https://github.com/catSirup/KorEDA.git

Cloning into 'KorEDA'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 19 (delta 7), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), done.


In [ ]:
%cd /content/KorEDA
# ! python 5장-1.py 
%run eda.py

/content/KorEDA


In [ ]:
%run eda.py

## train / test split 후 2번 label만 증강

In [ ]:
import random
import pickle
import re

wordnet = {}
with open("wordnet.pickle", "rb") as f:
	wordnet = pickle.load(f)

In [ ]:
from sklearn.model_selection import train_test_split
                                                         
train, test = train_test_split(data, test_size=0.25, random_state=0)

In [ ]:
train.Label.value_counts()

0    2566
1    2466
2     470
Name: Label, dtype: int64

In [ ]:
test.Label.value_counts()

0    906
1    783
2    145
Name: Label, dtype: int64

In [ ]:
# train 셋 내의 일반 문장만 선택
normal_train = train[train.Label==2]

In [ ]:
normal_train_rv = normal_train.Review

In [ ]:
len(normal_train_rv)
# 총 470개 -> 250개 sampling 해서 3개씩 증강

470

In [ ]:
normal_train_rv[:5]

4501                                      계속 녹음이 안되니까 보내죠
951                   삭막한 현실에서 코끼리에서 힐링하고 마음 치유 할 때가 많습니다
570      매일 명상을 통해 마음이 편안 해지고 고요함을 느끼며 또한 좋은 말씀을 통해 용기...
1041                    구매하고 싶은데요 각 영역마다 따로 구매를 해야 하는 건가요
2188     자동 결제는 무료 기간 동안 경험 해 보자는 생각에 일단 취소 했습니다 저에게 도...
Name: Review, dtype: object

In [ ]:
select = normal_train_rv.sample(250, random_state=100)

In [ ]:
aug = []
for rv in select:
    j = EDA(rv,
    alpha_sr=0.5, 
    alpha_ri=0.3, 
    alpha_rs=0.2, 
    p_rd=0.3, 
    num_aug=3)
    for r in j:
        aug.append(r)

In [ ]:
aug[:30]

['잠들기 직전에 생각나는 앱은 이 앱이 처음이 거든요 이 저는 현직 이 이 앱 개발자인데 이 저도 유저에게 이런 만족감을 주고 이 싶네요',
 '잠들기 직전에 생각나는 앱이 이 앱은 저는 거든요 처음이 주고 앱 개발자인데 저도 유저에게 이런 만족감을 현직 싶네요',
 '직전에 생각나는 앱이 처음이 저는 현직 개발자인데 저도 유저에게 이런 만족감을 주고 싶네요',
 ' 잠들기 직전에 생각나는 앱은 이 앱이 처음이 거든요 저는 현직 앱 개발자인데 저도 유저에게 이런 만족감을 주고 싶네요',
 '작년엔 가 명동 지하 식당에서 혜민 스님이 계셔서 약속 차 나오신 지 알았는데 인 곳에서 일하고 계셨을 줄이야 ㅎㅎ 암튼 좋은 콘텐츠 고맙습니다',
 '작년엔 가 명동 콘텐츠 식당에서 계셨을 스님이 줄이야 약속 좋은 나오신 건지 알았는데 그 곳에서 일하고 혜민 계셔서 ㅎㅎ 암튼 차 지하 고맙습니다',
 '건 작년엔 가 명동 지하 건 차 식당에서 건 혜민 스님이 계셔서 약속 차 나오신 건지 알았는데 그 곳에서 차 일하고 계셨을 줄이야 ㅎㅎ 암튼 차 좋은 콘텐츠 고맙습니다',
 ' 작년엔 가 명동 지하 식당에서 혜민 스님이 계셔서 약속 차 나오신 건지 알았는데 그 곳에서 일하고 계셨을 줄이야 ㅎㅎ 암튼 좋은 콘텐츠 고맙습니다',
 '명리 씨 우리 같이 명 상해요',
 '명리 씨 명 우리 같이 명 상해요',
 '명리 씨 우리 상해요 명 같이',
 ' 명리 씨 우리 같이 명 상해요',
 '화면이 꺼지면 음악도 꺼졌거든요 무료로 즐길 수 있는 캄 컨텐츠는 삼성 헬스 명 상 탭에 더 많은 것 같습니다',
 '화면이 꺼지면 명 음악도 꺼졌거든요 무료로 상 즐길 수 상 있는 캄 컨텐츠는 삼성 헬스 명 상 상 명 탭에 더 많은 것 같습니다',
 '화면이 꺼지면 음악도 꺼졌거든요 많은 즐길 수 무료로 캄 컨텐츠는 있는 헬스 명 상 탭에 더 삼성 것 같습니다',
 ' 화면이 꺼지면 음악도 꺼졌거든요 무료로 즐길 수 있는 캄 컨텐츠는 삼성 헬스 명 상 탭에 더 많은 것 같습니다',
 '다음 업

In [ ]:
len(aug) # 3개씩 증강해서 1000개 맞춤

1000

In [ ]:
# 한번 섞어주기
import random
random.seed(100)
random.shuffle(aug)

In [ ]:
aug[:5]

['Free trial을 취소 했는데 결제 예정 메일이 왔다',
 ' 결제 일이 14일 13일 지금 결제가 되었다는 문자가 왔네요',
 '느리지만 천천히 바뀌는 조금씩은 것',
 '랩을 사용하고 개월 씩 자동 결재 되던데 앱을 삭제 하면 유료',
 ' 녹음 된 모든 시간을 들으려 면 유료 결재 해야 하나요']

## 원래 Train과 다시 합치기

In [ ]:
# 2번 라벨 가진 DF로 변환
augment = pd.DataFrame({'Review':aug,
                        'Label': int(2)})

In [ ]:
# train과 다시 합치기
train1 = pd.concat([train, augment])

In [ ]:
train1.Label.value_counts()

0    2566
1    2466
2    1470
Name: Label, dtype: int64

In [ ]:
# 변환 안 된 문장이 있을 수 있으니 중복 제거
train1 = train1.drop_duplicates(['Review'])

In [ ]:
train1.Label.value_counts()

0    2563
1    2461
2    1201
Name: Label, dtype: int64

In [ ]:
dataset_train = []
for q, label in zip(train1['Review'], train1['Label'])  :
    data_2 = []
    data_2.append(q)
    data_2.append(str(label))

    dataset_train.append(data_2)

In [ ]:
dataset_test = []
for q, label in zip(test['Review'], test['Label'])  :
    data_3 = []
    data_3.append(q)
    data_3.append(str(label))

    dataset_test.append(data_3)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

6225
1834


# KoBert

## Model

In [ ]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
device = torch.device("cuda:0") 

#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

/content/KorEDA/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/KorEDA/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
device

device(type='cuda', index=0)

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/KorEDA/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
data_train[30]

(array([   2, 1042, 4977, 1844, 5859, 3084, 5439,  517, 6655, 1844, 5859,
        2613, 4998, 6999,  517,  493,  493,    3,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(18, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

## train / test epoch 10번

In [ ]:
train1 = []
test1 = []
loss1 = []


for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {:.4f} train acc {:.4f}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    train1.append(train_acc / (batch_id+1))
    loss1.append(loss)
    print("epoch {} train acc {:.4f}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    test1.append(test_acc / (batch_id+1))
    print("epoch {} test acc {:.4f}".format(e+1, test_acc / (batch_id+1)))
    print("")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


  0%|          | 0/98 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.0219 train acc 0.5156
epoch 1 train acc 0.6497


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 9 test acc 0.8442



  0%|          | 0/98 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.4680 train acc 0.8438
epoch 10 train acc 0.9231


  0%|          | 0/29 [00:00<?, ?it/s]

epoch 10 test acc 0.8437



In [ ]:
print(train1)
print(test1)
print(loss1)

[0.6497130102040817, 0.7452168367346939, 0.6387117346938775, 0.6532206632653061, 0.7366071428571429, 0.7700799069627852, 0.8111025660264105, 0.8737995198079231, 0.9007446728691476, 0.923141131452581]
[0.07953612479474549, 0.07953612479474549, 0.07953612479474549, 0.09192836617405584, 0.13423645320197045, 0.36224856321839083, 0.6795207307060755, 0.8029813218390804, 0.8442374794745484, 0.8436986863711001]
[tensor(0.1142, device='cuda:0', grad_fn=<NllLossBackward0>), tensor(0.0549, device='cuda:0', grad_fn=<NllLossBackward0>), tensor(0.0788, device='cuda:0', grad_fn=<NllLossBackward0>), tensor(0.3185, device='cuda:0', grad_fn=<NllLossBackward0>), tensor(0.1541, device='cuda:0', grad_fn=<NllLossBackward0>), tensor(0.7298, device='cuda:0', grad_fn=<NllLossBackward0>), tensor(0.5453, device='cuda:0', grad_fn=<NllLossBackward0>), tensor(0.6556, device='cuda:0', grad_fn=<NllLossBackward0>), tensor(0.5478, device='cuda:0', grad_fn=<NllLossBackward0>), tensor(0.7114, device='cuda:0', grad_fn=<Nl

## with epoch 30번

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 30
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/KorEDA/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
data_train[30]

(array([   2, 1042, 4977, 1844, 5859, 3084, 5439,  517, 6655, 1844, 5859,
        2613, 4998, 6999,  517,  493,  493,    3,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(18, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [ ]:
train2 = []
test2 = []
loss2 = []


for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {:.4f} train acc {:.4f}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    train2.append(train_acc / (batch_id+1))
    loss2.append(loss)
    print("epoch {} train acc {:.4f}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    test2.append(test_acc / (batch_id+1))
    print("epoch {} test acc {:.4f}".format(e+1, test_acc / (batch_id+1)))
    print("")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


  0%|          | 0/98 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.3247 train acc 0.1406
epoch 1 train acc 0.7374


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/29 [00:00<?, ?it/s]

...

epoch 29 test acc 0.8413



  0%|          | 0/98 [00:00<?, ?it/s]

epoch 30 batch id 1 loss 0.1780 train acc 0.9688
epoch 30 train acc 0.9928


  0%|          | 0/29 [00:00<?, ?it/s]

epoch 30 test acc 0.8413



In [ ]:
print(train2)
print(test2)
print(loss2)

[0.7374043367346939, 0.9250637755102041, 0.868781887755102, 0.8156887755102041, 0.8266900510204082, 0.8260522959183674, 0.7448229291716687, 0.9140625, 0.9092793367346939, 0.9221938775510204, 0.9418048469387755, 0.9433992346938775, 0.9579081632653061, 0.9673150510204082, 0.974968112244898, 0.9783163265306123, 0.9850127551020408, 0.9862882653061225, 0.9886798469387755, 0.9896364795918368, 0.9896364795918368, 0.9897959183673469, 0.9913903061224489, 0.9918686224489796, 0.9926658163265306, 0.9925063775510204, 0.9929846938775511, 0.9931441326530612, 0.9929846938775511, 0.9928252551020408]
[0.7167744252873562, 0.27360426929392445, 0.4019139983579639, 0.3811063218390805, 0.07953612479474549, 0.2363762315270936, 0.555008210180624, 0.445607553366174, 0.6318247126436782, 0.6342877668308703, 0.620279146141215, 0.6213310755336617, 0.7331691297208539, 0.7989275451559934, 0.8064963054187192, 0.8046233579638752, 0.8234554597701149, 0.8288947044334974, 0.8420823070607554, 0.8380285303776682, 0.83967056

# 증강 모델 라벨별 맞춘 수

In [ ]:
def calc_accuracy(X,Y, l):
    max_vals, max_indices = torch.max(X, 1)
    for i in max_indices:
      l.append(i)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
model.eval()
labels=[]
predict=[]
test_acc=0
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    for l in label:
      labels.append(l)
    label = label.long().to(device)
    
    out = model(token_ids, valid_length, segment_ids)
    test_acc += calc_accuracy(out, label, predict)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/29 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
cnt0 = 0
cnt1 = 0
cnt2 = 0
test_labels = []
predict_labels=[]

In [ ]:
pdt0 = 0
pdt1 = 0
pdt2 = 0

In [ ]:
for i in range(len(labels)):
  if labels[i] == 0:
    cnt0 +=1
    test_labels.append(0)
  elif labels[i] == 1:
    cnt1+=1
    test_labels.append(1)
  else:
    cnt2+=1
    test_labels.append(2)

In [ ]:
for i in range(len(predict)):
  if predict[i] == 0:
    pdt0+=1
    predict_labels.append(0)
  elif predict[i] == 1:
    pdt1+=1
    predict_labels.append(1)
  else:
    pdt2+=1
    predict_labels.append(2)

In [ ]:
print(cnt0, cnt1, cnt2)
print(pdt0, pdt1, pdt2)

906 783 145
865 792 177


In [ ]:
predict_df = pd.DataFrame({'Test Label' : test_labels,
              'Predict Label': predict_labels})

### 0 (부정) 예측 정확도

In [ ]:
print(f"0 총 개수: {len(predict_df[predict_df['Test Label']==0])}\n0 예측성공: {sum(predict_df[predict_df['Test Label']==0].iloc[:,1]==0)}")
print('0 정확도: ', round(791/906,4))

0 총 개수: 906
0 예측성공: 791
0 정확도:  0.8731


### 1 (긍정) 예측 정확도

In [ ]:
print(f"1 총 개수: {len(predict_df[predict_df['Test Label']==1])}\n1 예측성공: {sum(predict_df[predict_df['Test Label']==1].iloc[:,1]==1)}")
print('1 정확도: ', round(693/783,4))

1 총 개수: 783
1 예측성공: 693
1 정확도:  0.8851


### 2 (일반) 예측 정확도

In [ ]:
print(f"2 총 개수: {len(predict_df[predict_df['Test Label']==2])}\n2 예측성공: {sum(predict_df[predict_df['Test Label']==2].iloc[:,1]==2)}")
print('2 정확도: ', round(57/145,4))

2 총 개수: 145
2 예측성공: 57
2 정확도:  0.3931


In [ ]:
print(f"2를 1로 예측: {sum(predict_df[predict_df['Test Label']==2].iloc[:,1]==1)}")
print(f"2를 0로 예측: {sum(predict_df[predict_df['Test Label']==2].iloc[:,1]==0)}")
## 일반 문장을 긍정, 부정으로 골고루 잘못 판단함

2를 1로 예측: 23
2를 0로 예측: 29
